# Data Wrangling 

Project Goal: In this project, I aim to build models to predict the final price of homes in Ames, Iowa
based on 79 different given property features.
The data for this project has been retrieved from: https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview
***

In this section, I will take a look at the data to answer the following questions: How many data points do I have? What are the variables (column names/property features) and data types? What is the percentage of data points missing and for which variables? And last, but not least, is the final saleprice for all entities available?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Let's start with transforming the provided data description from a text file to a Pandas DataFrame so that I can easily access it throughout the rest of the project. 

In [2]:
# Creating loop to transfom data description txt file into list of lists
data_desc_file = '../cs2-housing/documents/data_description.txt'
data_desc_dict = []

with open(data_desc_file) as file:
    for line in file:
        if not line.isspace():
            if not line.startswith(" "):
                new_line = line.split(":")
                data_desc_dict.append([new_line[0].strip(), new_line[1].strip(), ''])
            else:
                data_desc_dict[-1][2] += line.replace('\n', '').replace('\t', ':')
                
# Creating pd dataframe for Data Dictionary, replacing blank space with NaN values
column_names = ['Name', 'Description', 'Notes']            
data_dict_df = pd.DataFrame(data_desc_dict,columns=column_names)
data_dict_df = data_dict_df.replace(r'^\s*$', np.nan, regex=True)
data_dict_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Name         79 non-null     object
 1   Description  79 non-null     object
 2   Notes        46 non-null     object
dtypes: object(3)
memory usage: 2.0+ KB


Now it's time read the data to a DataFrame and see how many data points and columns I'm dealing with. 

In [3]:
housing_raw = pd.read_csv('../cs2-housing/rawdata/train.csv')

In [4]:
housing_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [5]:
housing_raw.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


Looks like this data set has 1460 data points and 81 columns. Looking at the column names, it appears that the month and year of sale for each property has been split into two columns. Let's combine those to a single datetime column.

In [6]:
# Combining MoSold and YrSold to new Datetime column YrMoSold, dropping MoSold and YrSold Columns
housing_clean = housing_raw
housing_clean[['YrSold', 'MoSold']] = housing_clean[['YrSold', 'MoSold']].astype('str')
housing_clean['YrMoSold'] = housing_clean['YrSold'].str.cat(housing_clean['MoSold'],sep='-')
housing_clean['YrMoSold'] = pd.to_datetime(housing_clean['YrMoSold'], format='%Y-%m')
housing_clean = housing_clean.drop(labels=['MoSold', 'YrSold'], axis=1)

Now that I have combined the property closing dates into one column, let's take a look at how much data each column is missing:

In [7]:
# Count and percentage of missing data in columns
missing = pd.concat([housing_clean.isnull().sum(), 100 * housing_clean.isnull().mean().round(4)], axis=1)
missing.columns=['count', '%']
missing_values = missing.sort_values(by=['count', '%'], ascending=False)
cols_missing = missing_values.loc[missing_values['count'] != 0.00]
cols_missing

,count,%
PoolQC,1453,99.52
MiscFeature,1406,96.30
Alley,1369,93.77
Fence,1179,80.75
FireplaceQu,690,47.26
LotFrontage,259,17.74
GarageType,81,5.55
GarageYrBlt,81,5.55
GarageFinish,81,5.55
GarageQual,81,5.55


Looks like a lot of properties in Ames, Iowa don't have a pool, alley, or a fence. But what are the other miscellaneous features that a lot of the properties don't have?

In [8]:
# Miscellaneous features and their monetary values
misc_features = housing_clean[['MiscFeature', 'MiscVal']].dropna()
print(misc_features['MiscFeature'].unique())
print(misc_features)

['Shed' 'Gar2' 'Othr' 'TenC']
     MiscFeature  MiscVal
5           Shed      700
7           Shed      350
16          Shed      700
17          Shed      500
51          Shed      400
84          Shed      700
95          Shed      480
98          Shed      400
99          Shed      400
106         Shed      450
214         Shed      450
249         Shed      500
250         Shed      450
335         Shed      700
338         Shed      400
346         Gar2    15500
392         Shed     1200
439         Shed      800
499         Shed      480
502         Shed      400
510         Shed     2000
539         Shed     2000
589         Shed      600
611         Shed      500
626         Shed      600
634         Shed      600
705         Othr     3500
725         Shed      500
733         Shed      400
760         Shed      450
766         Shed      500
767         Shed     1300
786         Shed     1200
794         Shed      500
800         Shed      400
812         Shed       54
813     

Okay, Miscellaneous features, in this case, primarily refer to sheds and additional garages. Good to know!  
Now, I noticed that the column SalePrice was not included in the missing value counts! But before I get too happy about having all the data for that, let's take a look at the values in that column to make sure that the sale prices are truly all there.
To do so, I will take a look at the lowest sale prices as well as the condition they were sold under. 

In [9]:
# Lowest sales prices
lowest_final_price = housing_clean.loc[housing_clean.SalePrice < 50000]
lowest_final_price[['SaleCondition', 'SalePrice']]

,SaleCondition,SalePrice
30,Normal,40000
495,Abnorml,34900
533,Normal,39300
916,Abnorml,35311
968,Abnorml,37900


In [10]:
# checking against data dictionary to see what Sale Condition Abnormal means
row = data_dict_df[data_dict_df['Name'] == 'SaleCondition']
print(row.to_json())

{"Name":{"78":"SaleCondition"},"Description":{"78":"Condition of sale"},"Notes":{"78":"       Normal:Normal Sale       Abnorml:Abnormal Sale -  trade, foreclosure, short sale       AdjLand:Adjoining Land Purchase       Alloca:Allocation - two linked properties with separate deeds, typically condo with a garage unit:       Family:Sale between family members       Partial:Home was not completed when last assessed (associated with New Homes)"}}


Cool! All our properties have Sale prices!
Okay, then there's only one more thing I want to clean up before I'm done here. Remember the data dictionary I created in the beginning? All the columns in that data dictionary indicate categorical data. I will use the data dictionary to parse over my data to change the dtype of those columns to objects.

In [11]:
# Creating list of columns that should have categorical dtype
dtype_object = ['Id']
for index, row in data_dict_df.iterrows():
    if pd.notnull(row[2]):
        dtype_object.append(row[0])

# Casting correct dtypes for categorical data columns and creating new df for cleaned housing data
housing_dtypes_clean = housing_clean[dtype_object].astype('object')
housing_clean = housing_dtypes_clean.merge(housing_clean).drop_duplicates(keep='first').reset_index(drop=True)
housing_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Id             1460 non-null   object        
 1   MSSubClass     1460 non-null   object        
 2   MSZoning       1460 non-null   object        
 3   Street         1460 non-null   object        
 4   Alley          91 non-null     object        
 5   LotShape       1460 non-null   object        
 6   LandContour    1460 non-null   object        
 7   Utilities      1460 non-null   object        
 8   LotConfig      1460 non-null   object        
 9   LandSlope      1460 non-null   object        
 10  Neighborhood   1460 non-null   object        
 11  Condition1     1460 non-null   object        
 12  Condition2     1460 non-null   object        
 13  BldgType       1460 non-null   object        
 14  HouseStyle     1460 non-null   object        
 15  OverallQual    1460 n

There we go! Now I have 1460 data points with 79 property features to work with. 1 datetime column, 32 numerical columns, and 47 object columns. And the sale prices are available for all properties! Yay!
Time to save my cleaned data to csv files!

In [12]:
# Saving cleaned DFs to new csv files
data_dict_df.to_csv('../cs2-housing/documents/data_dictionary.csv')
housing_clean.to_csv('../cs2-housing/rawdata/cleaned_data.csv')

## __And off we go to do some EDA!__